In [105]:
import pandas as pd
import numpy as np

import keras as keras
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model

In [3]:
#import CCLE data
E_CCLE = pd.read_csv('E_CCLE.csv', index_col=0)
M_CCLE = pd.read_csv('M_CCLE.csv',index_col=0)


In [4]:
E_TCGA = pd.read_csv('E_TCGA.csv',index_col=0)

In [5]:
M_TCGA = pd.read_csv('M_TCGA.csv',index_col=0)

In [6]:
IC50 = pd.read_csv('IC50_CCLE_author.csv',index_col=0)

In [7]:
E_CCLE = E_CCLE.transpose()
E_CCLE.head()

,ETS1,CATIP,MIR34B,ADAMTS8,LINC01124,ZBTB20-AS4,LRP1B,MOGS,HOXC-AS3,OTOG,...,OBSCN,ALAS1,SCARNA8,MLST8,LACTB2,NFASC,PRR27,TBL1XR1,OR51G1,GRAMD1C
NIHOVCAR3,4.055716,0.263034,0.0,2.104337,0.250962,0.163499,0.871844,5.570159,0.641546,0.344828,...,3.874797,5.246408,0.000000,5.523248,5.481557,2.963474,0.014355,5.813012,0.0,2.786596
HL60,0.739848,0.042644,0.0,0.028569,0.000000,0.000000,0.000000,5.204376,0.000000,0.000000,...,0.137504,6.418359,0.000000,6.131343,3.266037,0.189034,0.000000,4.297925,0.0,0.014355
CACO2,3.117695,0.111031,0.0,0.042644,0.831877,0.000000,0.042644,5.580447,0.056584,0.286881,...,1.176323,5.414812,0.000000,5.115200,6.356672,0.378512,0.000000,3.730096,0.0,1.280956
HEL,7.598574,0.565597,0.0,0.056584,0.189034,0.000000,0.014355,5.250583,0.000000,0.176323,...,4.122673,5.782933,2.990955,6.146289,5.060047,0.111031,0.000000,4.605850,0.0,1.021480
HEL9217,8.145728,0.298658,0.0,0.014355,0.097611,0.070389,0.000000,4.954196,0.014355,0.000000,...,2.954196,5.481234,0.000000,5.722466,5.233811,0.124328,0.000000,5.445594,0.0,0.887525


In [8]:
M_CCLE = M_CCLE.transpose()
M_CCLE.head()

,TBX19,TMCO6,MKI67,CDH6,TRPC1,ETS1,CLSTN3,RGPD6,PLEKHN1,KRTAP20-1,...,FBXO28,SPRR2F,CD300LG,CYB5A,CYP4F2,TBL1XR1,PNRC2,KCNE2,SF3A2,GRAMD1C
201T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22RV1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2313287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
253J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253JBV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
M_TCGA = M_TCGA.transpose()
M_TCGA.head()

,TBX19,TMCO6,MKI67,CDH6,TRPC1,ETS1,CLSTN3,RGPD6,PLEKHN1,KRTAP20-1,...,FBXO28,SPRR2F,CD300LG,CYB5A,CYP4F2,TBL1XR1,PNRC2,KCNE2,SF3A2,GRAMD1C
0004d251-3f70-4395-b175-c94c2f5b1b81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000d566c-96c7-4f1c-b36e-fa2222467983,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0011a67b-1ba9-4a32-a6b8-7850759a38cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001887aa-36d0-463f-8bca-dec7043b4f2e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001944e5-af34-4061-9c09-bb9ea346f6fd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
E_TCGA = E_TCGA.transpose()
E_TCGA.head()

,ETS1,CATIP,MIR34B,ADAMTS8,LINC01124,ZBTB20-AS4,LRP1B,MOGS,HOXC-AS3,OTOG,...,OBSCN,ALAS1,SCARNA8,MLST8,LACTB2,NFASC,PRR27,TBL1XR1,OR51G1,GRAMD1C
0004d251-3f70-4395-b175-c94c2f5b1b81,16.027345,11.869594,-9.965784,7.584970,15.447471,-9.965784,-9.965784,17.577340,-9.965784,15.050699,...,10.857204,20.859636,-9.965784,16.890003,16.178120,16.362406,10.007029,17.149013,-9.965784,12.691308
000d566c-96c7-4f1c-b36e-fa2222467983,17.984969,13.958462,-9.965784,14.822123,11.078151,7.033434,10.751545,18.721833,-9.965784,5.285439,...,17.372627,17.944398,-9.965784,18.279923,15.043198,16.044458,7.257397,18.837905,-9.965784,14.983216
0011a67b-1ba9-4a32-a6b8-7850759a38cf,19.086699,9.744836,-9.965784,11.464546,12.231522,6.584978,6.857993,18.448399,1.585443,8.189829,...,15.780232,18.232641,5.554620,17.427968,17.220416,16.469769,-9.965784,19.250266,-9.965784,14.008429
001887aa-36d0-463f-8bca-dec7043b4f2e,17.034885,13.605248,-9.965784,10.784636,16.775842,6.569871,5.977303,18.404248,4.322000,6.658226,...,11.806953,21.763549,-9.965784,17.920615,18.067566,14.661110,10.832891,18.463832,-9.965784,15.740176
001944e5-af34-4061-9c09-bb9ea346f6fd,18.023949,10.115045,-9.965784,12.508290,-9.965784,5.584993,12.330917,18.694909,-9.965784,5.044438,...,14.592749,17.939281,6.584978,18.493242,17.248863,12.841171,-9.965784,20.048447,4.322000,15.648723


In [11]:
IC50.head()

,X22RV1,X2313287,X42MGBA,X5637,X639V,X647V,X697,X769P,X786O,X8305C,...,VCAP,VMCUB1,VMRCRCW,VMRCRCZ,WM115,WSUDLCL2,YAPC,YH13,YKG1,ZR7530
TL-2-105,4.6782,4.7229,4.1668,3.71110,4.4224,4.6946,1.19990,4.989000,4.1650,5.4715,...,4.2607,4.8047,4.53320,3.787200,3.5018,2.5862,5.5113,4.0055,4.61160,5.714937
TAK-715,4.7217,3.7747,5.5256,3.73420,4.9266,3.9165,1.60470,2.609200,3.1698,6.0290,...,4.6174,5.4785,4.14860,3.748600,5.1913,2.8146,5.7889,5.5701,3.70750,5.853699
CP466722,2.1995,2.9851,3.5194,1.80130,3.2444,2.0760,-0.86400,1.877600,2.1493,3.7663,...,1.8166,3.9553,2.80750,1.950200,1.9625,2.0836,3.6385,2.2756,2.41570,4.580076
BMS-345541,2.7886,3.4029,4.3641,2.67680,4.6885,3.3922,0.79615,1.491900,1.5246,5.1374,...,3.0436,4.6465,3.14600,3.087400,3.0976,2.4709,3.7157,3.5809,2.42940,4.263374
Genentech_Cpd_10,2.2161,2.8008,3.5257,-0.42507,2.1349,3.3500,-1.41520,-0.013295,-1.5783,5.3360,...,1.7366,2.5529,0.87061,-0.008367,1.8015,2.4950,5.6822,2.8236,0.91206,4.259721


In [12]:
#Train a gene expresion autoencoder with TCGA data (11284, 22374), Eenc (15,363, 1,024, 256, and 64)

In [50]:
encoding_dim = 64  # 64 floats -> compression of factor 343, assuming the input is 22374 floats

tumor_dim = E_TCGA.shape[1]
tumor_dim


22374

In [55]:
from keras import backend as K


In [149]:
K.clear_session()
layers = [1024,256,128,encoding_dim]
# this is our input placeholder
input_tumor = Input(shape=(tumor_dim,))
# "encoded" is the encoded representation of the input
for i,layer in enumerate(layers):
    encoded = Dense(layer, activation='relu')(input_tumor if i == 0 else encoded)
    encoded = BatchNormalization()(encoded)
for i,layer in enumerate(layers[-2::-1]):
    decoded = Dense(layer, activation='relu')(encoded if i == 0 else decoded)
    decoded = BatchNormalization()(decoded)
out = Dense(tumor_dim, activation='linear')(decoded)


In [142]:
layers[-2::-1]

[128, 256, 1024]

In [150]:
# this model maps an input to its reconstruction
autoencoder = Model(input_tumor, out)
# this model maps an input to its encoded representation
encoder = Model(input_tumor, encoded)

In [151]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 22374)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              22912000  
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
__________

In [152]:
# create a placeholder for an encoded (64-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = Model(encoded_input, decoder_layer(encoded_input))
#decoder = Model(encoded_input, decoder_layer(encoded_input))

In [153]:
adam = keras.optimizers.Adam(lr=1e-1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
autoencoder.compile(optimizer= adam, loss='mean_squared_error')


In [111]:
len(list(E_TCGA.index.values))

11284

In [97]:
E_TCGA_norm = (E_TCGA-E_TCGA.mean())/E_TCGA.std()


In [98]:
E_TCGA_norm['ETS1'].mean(), E_TCGA_norm['ETS1'].std()

(-5.314180306170729e-14, 1.0000000000000002)

In [99]:
E_TCGA_norm.head()

,ETS1,CATIP,MIR34B,ADAMTS8,LINC01124,ZBTB20-AS4,LRP1B,MOGS,HOXC-AS3,OTOG,...,OBSCN,ALAS1,SCARNA8,MLST8,LACTB2,NFASC,PRR27,TBL1XR1,OR51G1,GRAMD1C
0004d251-3f70-4395-b175-c94c2f5b1b81,-1.653684,0.042970,-0.290539,-1.785548,1.049974,-1.941253,-3.215100,-1.057580,-1.754631,1.394018,...,-3.251632,2.584364,-1.762178,-1.161979,-0.163461,0.227200,1.428210,-2.079530,-0.259656,-1.316218
000d566c-96c7-4f1c-b36e-fa2222467983,-0.357614,0.974921,-0.290539,0.919611,0.272991,0.335058,0.142264,0.344208,-1.754631,0.078873,...,0.353153,-0.201507,-1.762178,0.395931,-0.992445,0.089714,1.099940,-0.581491,-0.259656,-0.056558
0011a67b-1ba9-4a32-a6b8-7850759a38cf,0.371799,-0.904993,-0.290539,-0.335410,0.478091,0.275006,-0.488709,0.009302,-0.445045,0.470024,...,-0.527870,0.073945,0.337997,-0.558995,0.597866,0.273625,-0.956283,-0.215730,-0.259656,-0.592313
001887aa-36d0-463f-8bca-dec7043b4f2e,-0.986629,0.817334,-0.290539,-0.589552,1.286194,0.272983,-0.631430,-0.044774,-0.134796,0.263754,...,-2.726165,3.448166,-1.762178,-0.006805,1.216651,-0.508468,1.526808,-0.913292,-0.259656,0.359476
001944e5-af34-4061-9c09-bb9ea346f6fd,-0.331807,-0.739824,-0.290539,0.054729,-3.469190,0.141101,0.398210,0.311231,-1.754631,0.046416,...,-1.184869,-0.206397,0.477422,0.635031,0.618645,-1.295439,-0.956283,0.492252,3.412015,0.309212


In [100]:
#reserve a random sample of 2000 samples to validate the model
from random import sample 
valid_idx = sample(list(E_TCGA_norm.index.values),2000 )
x_valid =  E_TCGA_norm.loc[valid_idx, :]
x_train = E_TCGA_norm.drop(valid_idx, axis=0)
print (x_train.shape) 
print (x_valid.shape)

(9284, 22374)
(2000, 22374)


In [154]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_valid, x_valid))

Train on 9284 samples, validate on 2000 samples
Epoch 1/50
9284/9284 [==============================] - 18s 2ms/step - loss: 84.4921 - val_loss: 114.5008
Epoch 2/50
9284/9284 [==============================] - 17s 2ms/step - loss: 1.1884 - val_loss: 4.6348
Epoch 3/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.8152 - val_loss: 1.6592
Epoch 4/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.8472 - val_loss: 0.9902
Epoch 5/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.9209 - val_loss: 1.0587
Epoch 6/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.8697 - val_loss: 0.8907
Epoch 7/50
9284/9284 [==============================] - 17s 2ms/step - loss: 0.9444 - val_loss: 0.9747
Epoch 8/50
9284/9284 [==============================] - 17s 2ms/step - loss: 1.1603 - val_loss: 1.1292
Epoch 9/50
9284/9284 [==============================] - 17s 2ms/step - loss: 1.4001 - val_loss: 1.0586
Epoch 10/50
9284/9284 

KeyboardInterrupt: 

In [170]:
autoencoder.predict(x_valid)[1][:100]

array([ 0.6230566 ,  0.498782  , -0.22708845,  0.89156705, -0.10609552,
        0.86724305,  1.1581866 , -0.18311933, -0.06177109,  0.4086734 ,
        0.13769686,  0.19945411,  0.4054064 , -0.3328606 ,  1.0980283 ,
       -0.2619667 ,  0.41459733,  1.2674783 ,  0.2597496 ,  0.04445371,
       -0.10112071, -0.24648747,  0.25986472,  0.05560642,  0.5811808 ,
       -0.37404874,  1.0791408 , -0.50814486,  0.2442047 , -0.07099935,
        0.77779895,  0.6040886 , -0.47814316,  0.5763875 ,  0.7870445 ,
        0.20543134,  0.40919527, -0.27585834, -0.09101793,  0.5033603 ,
       -0.24176303,  0.48075125,  0.58626544,  0.99133587,  0.224314  ,
        1.004927  ,  0.4698176 ,  0.6379076 , -0.5719075 ,  0.22008356,
        1.0446115 ,  0.5521552 ,  0.85412467,  0.5595674 ,  1.2740711 ,
        0.3791816 ,  0.41732556,  0.8499137 ,  0.52217406,  0.10743958,
       -0.18150538,  0.85413617, -0.24457785, -0.00676137,  0.999386  ,
        1.00017   , -0.06450766,  0.12492704,  0.61199373,  0.54

In [171]:
x_valid.iloc[1,:100].values

array([ 0.67718697, -1.04339185, -0.29053882,  0.26404419, -0.23029879,
        0.55547471,  1.34184602,  0.35151323,  0.65693629,  0.20849949,
        0.19142875, -0.51997594,  1.20088052, -0.33911915,  0.87400657,
       -0.1366423 ,  0.48494617,  0.656754  , -0.4364245 ,  0.4065648 ,
        1.60313842,  0.30206363, -0.35023402, -0.90842213,  0.01980432,
       -1.60601085,  1.42372556, -0.1578078 ,  0.99930182,  0.28730806,
        0.88188019, -0.61823733, -0.31595982,  1.78773218, -0.16947625,
        0.36826668,  0.7683447 , -0.68204837,  0.66140369,  1.3206525 ,
       -0.25045018, -0.20661205,  1.12925182,  0.59379179,  0.12121542,
        1.52782294,  0.40672323,  0.08242651,  0.74986737, -0.29005026,
        0.4088975 ,  1.188109  ,  0.73687811,  0.62853628,  2.2942857 ,
        0.65839657,  0.45383037,  1.07844278,  1.30204107,  0.05437446,
       -1.22913346,  1.01918223, -0.48004211,  0.18501986,  0.8773076 ,
        1.13863572,  3.2257347 , -0.0806405 ,  0.74198204,  0.54